In [1]:
import pandas as pd
import numpy as np
PATH_PRE = "./Appolo/A_Dataset/highccgroup/PreDataset.csv"
PATH_PRICE = "./raw_data/kerosene_price.csv"
PATH_SAVE = "./Appolo/A_Dataset/highccgroup/Train_Test.csv"

In [18]:
df = pd.read_csv(PATH_PRE)
df_price = pd.read_csv(PATH_PRICE)
df["supplied_at"] = pd.to_datetime(df["supplied_at"], format="%Y-%m-%d %X")
df_price["date"] = pd.to_datetime(df_price["date"], format="%Y-%m")

In [3]:
df.head()

,customer_id,supplied_at,volume,size,sum,mean,matemp,mltemp,mhtemp,mwind,msnowacc,msnow,msun,mhumidi,mean_consumption,interval
0,251.0,2020-11-20 08:26:00,109.8,198,1653.9,11.729787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,251.0,2020-12-03 10:20:00,124.3,198,1653.9,11.729787,-2.300000,-0.138462,-4.353846,2.730769,1.000000,0.769231,1.976923,67.000000,9.561538,13.0
2,251.0,2020-12-18 09:32:00,177.9,198,1653.9,11.729787,0.100000,2.486667,-2.766667,2.980000,1.333333,0.800000,3.800000,62.466667,11.860000,15.0
3,251.0,2020-12-29 13:51:00,144.6,198,1653.9,11.729787,2.745455,5.863636,-0.290909,3.036364,11.727273,2.545455,2.390909,68.454545,13.145455,11.0
4,251.0,2021-01-13 16:15:00,248.7,198,1653.9,11.729787,6.440000,10.193333,3.253333,2.340000,28.066667,4.266667,3.853333,69.066667,16.580000,15.0


In [ ]:
# Using 椿データ we need to drop purepose
df.pop("purpose")

In [5]:
dataset = df.copy()

In [12]:

dataset_final = dataset.copy()
for i in dataset.index:
    if np.isnan(dataset.loc[i, "matemp"]):
        dataset_final.drop(i, inplace=True)
dataset_final.reset_index(inplace=True, drop=True)

In [13]:
dataset_final.head()

,customer_id,supplied_at,volume,size,sum,mean,matemp,mltemp,mhtemp,mwind,msnowacc,msnow,msun,mhumidi,mean_consumption,interval
0,251.0,2020-12-03 10:20:00,124.3,198,1653.9,11.729787,-2.300000,-0.138462,-4.353846,2.730769,1.000000,0.769231,1.976923,67.000000,9.561538,13.0
1,251.0,2020-12-18 09:32:00,177.9,198,1653.9,11.729787,0.100000,2.486667,-2.766667,2.980000,1.333333,0.800000,3.800000,62.466667,11.860000,15.0
2,251.0,2020-12-29 13:51:00,144.6,198,1653.9,11.729787,2.745455,5.863636,-0.290909,3.036364,11.727273,2.545455,2.390909,68.454545,13.145455,11.0
3,251.0,2021-01-13 16:15:00,248.7,198,1653.9,11.729787,6.440000,10.193333,3.253333,2.340000,28.066667,4.266667,3.853333,69.066667,16.580000,15.0
4,251.0,2021-01-28 08:15:00,199.6,198,1653.9,11.729787,3.160000,6.986667,-0.220000,3.560000,45.200000,3.933333,3.166667,71.800000,13.306667,15.0


In [14]:
len(dataset_final)

16229

In [15]:
dataset_final["last_year_mean"] = ""
dataset_final["last_year_sum"] = ""
unique_id = pd.unique(dataset_final["customer_id"])

# add last year data
for id in unique_id:
    df_id = dataset_final[dataset_final["customer_id"]==id]
    df_id.reset_index(inplace=True)
    for i in range(len(df_id)-1):
        if df_id.loc[i+1, "supplied_at"].month - df_id.loc[i, "supplied_at"].month > 6:
            for j in range(i, len(df_id)-1):
                index = df_id.loc[j, "index"]
                if(df_id.loc[j + 1, "supplied_at"].month - df_id.loc[j, "supplied_at"].month < 3):
                    dataset_final.loc[index, "last_year_mean"] =  df_id.loc[i, "mean"]
                    dataset_final.loc[index, "last_year_sum"] =  df_id.loc[i, "sum"]

In [16]:
len(dataset_final)

16229

In [17]:
# drop data that doesn't contain last year data
for i in dataset_final.index:
    if not isinstance(dataset_final.loc[i, "last_year_mean"], float):
        dataset_final.drop(i, inplace=True)
        
dataset_final.reset_index(inplace=True, drop=True)

In [19]:
# add price data
dataset_final["price"] = ""

for i in dataset_final.index:
    for j in df_price.index:
        if (dataset_final.loc[i, "supplied_at"].year == df_price.loc[j, "date"].year and 
        dataset_final.loc[i, "supplied_at"].month == df_price.loc[j, "date"].month):
            dataset_final.loc[i, "price"] = df_price.loc[j, "price"]

In [27]:
# drop data that doesn't contain last year data
for i in dataset_final.index:
    if not isinstance(dataset_final.loc[i, "price"], np.int64):
        print(i)

In [34]:
Traing_Test = dataset_final
Traing_Test.to_csv(PATH_SAVE, index=False)

In [35]:
for i in dataset_final.index:
    if np.isnan(dataset_final.loc[i, "last_year_mean"]):
        print(i)